In [32]:
import re

labels = []
data = []
with open("/home/user/바탕화면/programming/data_science/naive_bayes/SMSSpamCollection", "r") as f:
    for line in f:
        label, txt = line.split("\t")
        labels.append(label)
        
        data.append((label, set(re.findall("[a-z0-9_]+", txt.lower()))))
        

In [33]:
import random
random.shuffle(data)

n_train = int(len(data) * 0.8)

train = data[:n_train]
test = data[n_train:]

In [34]:
import math
alpha = 1e-4

n_ham = sum(1 for label, txt in train if label == 'ham')
n_spam = sum(1 for label, txt in train if label == 'spam')


prior_ham = math.log((n_ham + alpha) / (n_train + 2*alpha))
prior_spam = math.log((n_spam + alpha) / (n_train + 2*alpha)) 

prior_ham, prior_spam

(-0.1452941678165359, -2.000762486915931)

In [ ]:
from collections import defaultdict

word_ham = defaultdict(lambda : 0)
word_spam = defaultdict(lambda : 0)

for label, txt in train:
    for word in txt:
        if label == 'ham':
            word_ham[word] += 1
            
        else:
            word_spam[word] += 1
            
like_ham = defaultdict(lambda : math.log(alpha / (n_ham + alpha)))
like_spam = defaultdict(lambda : math.log(alpha / (n_spam + alpha)))

for word, cnt in word_ham.items():
    like_ham[word] = math.log((cnt + alpha) / (n_ham + alpha))
    
for word, cnt in word_spam.items():
    like_spam[word] = math.log((cnt + alpha) / (n_spam + alpha))


In [36]:
def predict(bow):
    ham_score = prior_ham
    spam_score = prior_spam
    for word in bow:
        ham_score += like_ham[word]
        spam_score += like_spam[word]
        
    if ham_score > spam_score:
        return 'ham'
    else:
        return 'spam'
    
    
        

In [37]:
test

[('ham', {'call', 'close', 'i', 'k', 'll', 'm', 'when', 'you'}),
 ('ham',
  {'as',
   'at',
   'before',
   'can',
   'class',
   'eve',
   'get',
   'good',
   'have',
   'if',
   'library',
   'not',
   'ok',
   'point',
   'see',
   'some',
   'sure',
   'tho',
   'time',
   'to',
   'try',
   'what',
   'will',
   'you'}),
 ('ham',
  {'already',
   'are',
   'baby',
   'back',
   'belly',
   'computer',
   'did',
   'filling',
   'get',
   'goes',
   'hello',
   'how',
   'i',
   'it',
   'loverboy',
   'miss',
   'mom',
   'now',
   's',
   'setting',
   'sighs',
   'the',
   'to',
   'up',
   'you',
   'your'}),
 ('ham',
  {'baby',
   'cant',
   'i',
   'real',
   'taste',
   'thank',
   'the',
   'thing',
   'to',
   'wait',
   'you'}),
 ('ham',
  {'break',
   'come',
   'fr',
   'get',
   'how',
   'i',
   'my',
   'n',
   'no',
   'one',
   'out',
   'stuff',
   'time'}),
 ('ham',
  {'a',
   'all',
   'always',
   'and',
   'as',
   'attributed',
   'come',
   'coming',
   'da

In [38]:
# accuracy = 맞은 데이터 수 / 전체 데이터 수
# precision? = tp / (tp + fp)
# 넌 던한테 잘해 게이게이게이야 트리플 게이

tp, fp, tn, fn = 0, 0, 0, 0

for label, txt in test:
    result = predict(txt)
    if result == 'spam' and label == 'spam':
        tp += 1
    elif result == 'spam' and label == 'ham':
        fp += 1
    elif result == 'ham' and label == 'ham':
        tn += 1
    elif result == 'ham' and label == 'spam':
        fn += 1
        
tp, fp, tn, fn 

(139, 33, 938, 5)

In [39]:
accuracy = (tp + tn) / (tp + tn + fp + fn)
recall = tp / (tp + fn)
precision = tp / (tp + fp)
f1 = 2 * recall * precision / (recall + precision)

print(accuracy, recall, precision, f1)

0.9659192825112107 0.9652777777777778 0.8081395348837209 0.879746835443038
